# Word embeddings
Word embeddings give us a way to use an efficient, dense representation in which similar words have a similar encoding.

Instead of specifying the values for the embedding manually, they are trainable parameters (weights learned by the model during training, in the same way a model learns weights for a dense layer).

Another way to think of an embedding is as "lookup table". After these weights have been learned, you can encode each word by looking up the dense vector it corresponds to in the table.



In [1]:
import io
import os
import re
import shutil
import string
import tensorflow as tf

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.layers import TextVectorization

In [2]:
url = "https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"

dataset = tf.keras.utils.get_file("aclImdb_v1.tar.gz", url,
                                  untar=True, cache_dir='.',
                                  cache_subdir='')

dataset_dir = os.path.join(os.path.dirname(dataset), 'aclImdb')
os.listdir(dataset_dir)

['.DS_Store', 'imdbEr.txt', 'test', 'imdb.vocab', 'README', 'train']

In [3]:
train_dir = os.path.join(dataset_dir, 'train')
os.listdir(train_dir)

['urls_unsup.txt',
 '.DS_Store',
 'neg',
 'urls_pos.txt',
 'unsup',
 'urls_neg.txt',
 'pos',
 'unsupBow.feat',
 'labeledBow.feat']

In [4]:
remove_dir = os.path.join(train_dir, 'unsup')
shutil.rmtree(remove_dir)

In [5]:
batch_size = 1024
seed = 123
train_ds = tf.keras.utils.text_dataset_from_directory(
  'aclImdb/train/',
  batch_size=batch_size,
  validation_split=0.2,
  subset='training',
  seed=seed
)
val_ds = tf.keras.utils.text_dataset_from_directory(
  'aclImdb/train/',
  batch_size=batch_size,
  validation_split=0.2,
  subset='validation',
  seed=seed
)

Found 25000 files belonging to 2 classes.
Using 20000 files for training.
Found 25000 files belonging to 2 classes.
Using 5000 files for validation.


In [6]:
for text_batch, label_batch in train_ds.take(1):
  for i in range(3):
    print(label_batch[i].numpy(), text_batch[i].numpy())

0 b"Oh My God! Please, for the love of all that is holy, Do Not Watch This Movie! It it 82 minutes of my life I will never get back. Sure, I could have stopped watching half way through. But I thought it might get better. It Didn't. Anyone who actually enjoyed this movie is one seriously sick and twisted individual. No wonder us Australians/New Zealanders have a terrible reputation when it comes to making movies. Everything about this movie is horrible, from the acting to the editing. I don't even normally write reviews on here, but in this case I'll make an exception. I only wish someone had of warned me before I hired this catastrophe"
1 b'This movie is SOOOO funny!!! The acting is WONDERFUL, the Ramones are sexy, the jokes are subtle, and the plot is just what every high schooler dreams of doing to his/her school. I absolutely loved the soundtrack as well as the carefully placed cynicism. If you like monty python, You will love this film. This movie is a tad bit "grease"esk (without

2022-07-13 20:56:32.562003: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


In [7]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [8]:
# Embed a 1,000 word vocabulary into 5 dimensions.
embedding_layer = tf.keras.layers.Embedding(1000, 5)

In [9]:
result = embedding_layer(tf.constant([1, 2, 3]))
result.numpy()

array([[-0.04515788, -0.04449826, -0.03580015,  0.00590343,  0.02507892],
       [ 0.01797258, -0.03561745,  0.04607553,  0.00401322, -0.02356991],
       [ 0.04758969,  0.02529616,  0.02945591,  0.01772108,  0.0419887 ]],
      dtype=float32)

When given a batch of sequences as input, an embedding layer returns a 3D floating point tensor, of shape `(samples, sequence_length, embedding_dimensionality)`. To convert from this sequence of variable length to a fixed representation there are a variety of standard approaches. You could use an RNN, Attention, or pooling layer before passing it to a Dense layer.

In [10]:
result = embedding_layer(tf.constant([[0, 1, 2], [3, 4, 5]]))
result.shape

TensorShape([2, 3, 5])

In [11]:
def custom_standardization(input_data):
  lowercase = tf.strings.lower(input_data)
  stripped_html = tf.strings.regex_replace(lowercase, '<br />', ' ')
  return tf.strings.regex_replace(
    stripped_html,
    '[%s]' % re.escape(string.punctuation), ''
  )

vocab_size = 10000
sequence_length = 100

vectorize_layer = TextVectorization(
  standardize=custom_standardization,
  max_tokens=vocab_size,
  output_mode='int',
  output_sequence_length=sequence_length
)

text_ds = train_ds.map(lambda x, y: x)
vectorize_layer.adapt(text_ds)

In [12]:
embedding_dim = 16

model = Sequential([
  vectorize_layer,
  Embedding(vocab_size, embedding_dim, name='embedding'),
  GlobalAveragePooling1D(),
  Dense(16, activation='relu'),
  Dense(1)
])

In [13]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir='logs')

In [14]:
model.compile(
  optimizer='adam',
  loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
  metrics=['accuracy']
)

In [15]:
model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=15,
  callbacks=[tensorboard_callback]
)

Epoch 1/15
20/20 [==============================] - 1s 45ms/step - loss: 0.6918 - accuracy: 0.5028 - val_loss: 0.6898 - val_accuracy: 0.4886
Epoch 2/15
20/20 [==============================] - 1s 37ms/step - loss: 0.6872 - accuracy: 0.5028 - val_loss: 0.6841 - val_accuracy: 0.4886
Epoch 3/15
20/20 [==============================] - 1s 37ms/step - loss: 0.6794 - accuracy: 0.5028 - val_loss: 0.6744 - val_accuracy: 0.4886
Epoch 4/15
20/20 [==============================] - 1s 37ms/step - loss: 0.6670 - accuracy: 0.5028 - val_loss: 0.6602 - val_accuracy: 0.4886
Epoch 5/15
20/20 [==============================] - 1s 37ms/step - loss: 0.6495 - accuracy: 0.5028 - val_loss: 0.6412 - val_accuracy: 0.4886
Epoch 6/15
20/20 [==============================] - 1s 37ms/step - loss: 0.6266 - accuracy: 0.5131 - val_loss: 0.6178 - val_accuracy: 0.5256
Epoch 7/15
20/20 [==============================] - 1s 38ms/step - loss: 0.5985 - accuracy: 0.5836 - val_loss: 0.5905 - val_accuracy: 0.5956
Epoch 8/15
20

In [16]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization (TextVec  (None, 100)              0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, 100, 16)           160000    
                                                                 
 global_average_pooling1d (G  (None, 16)               0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 16)                272       
                                                                 
 dense_1 (Dense)             (None, 1)                 17        
                                                                 
Total params: 160,289
Trainable params: 160,289
Non-trai

In [17]:
%load_ext tensorboard
%tensorboard --logdir logs

In [18]:
weights = model.get_layer('embedding').get_weights()[0]
vocab = vectorize_layer.get_vocabulary()

In [20]:
out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('metadata.tsv', 'w', encoding='utf-8')

for index, word in enumerate(vocab):
  if index == 0:
    continue
  vec = weights[index]
  out_v.write('\t'.join([str(x) for x in vec]) + '\n')
  out_m.write(word + '\n')
out_v.close()
out_m.close()